In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from datetime import datetime, timedelta

In [2]:
#Read the training set, make the time column index and set the frequency to 'hourly' 

train_data = pd.read_csv('/Users/iremaltiner/Desktop/ErdosProject/Bootcamp Project/train-val-test/ordered_train_set.csv',
                        parse_dates=['time','date'])
train_data = train_data.drop(['Unnamed: 0.1','Unnamed: 0.2', 'Unnamed: 0'],axis=1)
train_data = train_data.set_index('time')
train_data.index = pd.DatetimeIndex(train_data.index).to_period('H')

In [3]:
#Read the test set, make the time column index and set the frequency to 'hourly' 

test_data = pd.read_csv('/Users/iremaltiner/Desktop/ErdosProject/Bootcamp Project/train-val-test/ordered_test_set.csv',
                        parse_dates=['time','date'])
test_data = test_data.drop(['Unnamed: 0.1','Unnamed: 0.2', 'Unnamed: 0'],axis=1)
test_data = test_data.set_index('time')
test_data.index = pd.DatetimeIndex(test_data.index).to_period('H') 

In [4]:
#Read the validation set, make the time column index and set the frequency to 'hourly' 

val_data = pd.read_csv('/Users/iremaltiner/Desktop/ErdosProject/Bootcamp Project/train-val-test/ordered_seasonal_validation_set.csv',
                        parse_dates=['time','date'])
val_data = val_data.drop(['Unnamed: 0.1','Unnamed: 0.2', 'Unnamed: 0'],axis=1)
val_data = val_data.set_index('time')
val_data.index = pd.DatetimeIndex(val_data.index).to_period('H')

In [5]:
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.api import SARIMAX

In [13]:
#RMSE of the baseline model on the validation and test sets

baseline_test_rmse = np.sqrt(mean_squared_error(test_data['DA_price'],test_data['DA_price(t-1D)']))
baseline_val_rmse = np.sqrt(mean_squared_error(val_data['DA_price'],val_data['DA_price(t-1D)']))
print(f'Test RMSE of the baseline model: {baseline_test_rmse}\nValidation RMSE of the baseline model: {baseline_val_rmse}')

Test RMSE of the baseline model: 20.50418041624876
Validation RMSE of the baseline model: 25.956556020205632


In [ ]:
f1 = ['DA_price(t-1D)', 'DA_price(t-2D)',
       'DA_price(t-3D)', 'DA_price(t-4D)', 'DA_price(t-5D)', 'avg_DA_price(h-24)']
f2 = ['DA_price(t-1D)', 'DA_price(t-2D)','avg_DA_price(h-24)']
f3 = ['DA_price(t-1D)', 'DA_price(t-2D)',
       'DA_price(t-3D)', 'DA_price(t-4D)', 'DA_price(t-5D)', 'avg_DA_price(h-24)', 'DA_price(t-6D)',
     'DA_price(t-7D)', 'actual_load']

rmse = dict()
mylist = [f1, f2, f3, 0]
i = 1
for item in mylist:
    if item == 0:
        arima = SARIMAX(endog=train_data['DA_price'],
                   order=(24, 1, 7))
        fitted_arima = arima.fit(maxiter=200)
        val_obs = fitted_arima.apply(endog = val_data['DA_price'], refit=False)
        preds = val_obs.predict(0,len(val_data)-1,dynamic=False)
        rmse[f'No exog features rmse'] = np.sqrt(mean_squared_error(val_data['DA_price'],preds))
    else:
        arima = SARIMAX(endog=train_data['DA_price'], exog=train_data[item],
                       order=(24, 1, 7))
        fitted_arima = arima.fit(maxiter=200)
        val_obs = fitted_arima.apply(endog = val_data['DA_price'], exog=val_data[item], refit=False)
        preds = val_obs.predict(0,len(val_data)-1, dynamic=False)
        rmse[f'f{i} rmse'] = np.sqrt(mean_squared_error(val_data['DA_price'],preds))
        i += 1

In [9]:
#Best set of features according to the result in the previous cell is f1. 

f1 = ['DA_price(t-1D)', 'DA_price(t-2D)','DA_price(t-3D)', 'DA_price(t-4D)', 'DA_price(t-5D)', 'avg_DA_price(h-24)']

arima = SARIMAX(endog=train_data['DA_price'],exog=train_data[f1], order=(24, 1, 7)).fit(maxiter=200)

/usr/local/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           38     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.35749D+00    |proj g|=  8.46894D-02


 This problem is unconstrained.



At iterate    5    f=  3.33478D+00    |proj g|=  2.64852D-03

At iterate   10    f=  3.33375D+00    |proj g|=  6.75422D-03

At iterate   15    f=  3.33338D+00    |proj g|=  4.04958D-03

At iterate   20    f=  3.33298D+00    |proj g|=  4.66998D-03

At iterate   25    f=  3.33288D+00    |proj g|=  1.16106D-03

At iterate   30    f=  3.33262D+00    |proj g|=  5.64915D-03

At iterate   35    f=  3.33067D+00    |proj g|=  1.26478D-02

At iterate   40    f=  3.32825D+00    |proj g|=  6.75428D-03

At iterate   45    f=  3.32658D+00    |proj g|=  2.01712D-02

At iterate   50    f=  3.32585D+00    |proj g|=  3.48248D-03

At iterate   55    f=  3.32577D+00    |proj g|=  1.43104D-03

At iterate   60    f=  3.32570D+00    |proj g|=  2.64657D-03

At iterate   65    f=  3.32556D+00    |proj g|=  3.88024D-03

At iterate   70    f=  3.32542D+00    |proj g|=  1.95627D-03

At iterate   75    f=  3.32537D+00    |proj g|=  1.74555D-03

At iterate   80    f=  3.32534D+00    |proj g|=  1.13323D-03

At iter

/usr/local/lib/python3.9/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [21]:
#val_obs and test_obs are the new results objects that are created using the parameters computed from the training set
#They contain results of only validation and test sets respectively.  
#val_preds and test_preds are predictions of arima on the validation and test set respectively. 

val_obs = arima.apply(endog = val_data['DA_price'], exog=val_data[f1], refit=False)
val_preds = val_obs.predict(0,len(val_data)-1, dynamic=False)
arima_val_rmse = np.sqrt(mean_squared_error(val_data['DA_price'],val_preds))

test_obs = arima.apply(endog = test_data['DA_price'], exog=test_data[f1], refit=False)
test_preds = test_obs.predict(0,len(test_data)-1, dynamic=False)
arima_test_rmse = np.sqrt(mean_squared_error(test_data['DA_price'],test_preds))
arima_test_rmse

print(f'Validation RMSE of Arima: {arima_val_rmse}\nTest RMSE of Arima: {arima_test_rmse}')

Validation RMSE of Arima: 7.197967660129694
Test RMSE of Arima: 7.213072116175197


In [23]:
arima.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:               DA_price   No. Observations:                16118
Model:              SARIMAX(24, 1, 7)   Log Likelihood              -53592.174
Date:                Mon, 27 Nov 2023   AIC                         107260.349
Time:                        21:10:14   BIC                         107552.479
Sample:                    10-27-2020   HQIC                        107356.933
                         - 08-29-2022                                         
Covariance Type:                  opg                                         
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
DA_price(t-1D)         0.2799      0.006     48.776      0.000       0.269       0.291
DA_price(t-2D)         0.1236      0.004     34.980      0.000       0.117       0.131
DA_price(t-3D)         0.1133      0.003     37.112      0.000       0.107       0.119
DA_price(t-4D)         0.0938      0.003     31.073      0.000       0.088       0.100
DA_price(t-5D)         0.1357      0.003     43.439      0.000       0.130       0.142
avg_DA_price(h-24)    -0.0203      0.006     -3.184      0.001      -0.033      -0.008
ar.L1                 -0.2070      0.082     -2.512      0.012      -0.369      -0.045
ar.L2                  0.6592      0.054     12.271      0.000       0.554       0.764
ar.L3                  0.4670      0.035     13.404      0.000       0.399       0.535
ar.L4                 -0.0461      0.055     -0.840      0.401      -0.154       0.062
ar.L5                 -0.6563      0.046    -14.192      0.000      -0.747      -0.566
ar.L6                  0.2504      0.034      7.283      0.000       0.183       0.318
ar.L7                  0.2593      0.053      4.894      0.000       0.155       0.363
ar.L8                  0.0076      0.007      1.036      0.300      -0.007       0.022
ar.L9                  0.0485      0.006      7.911      0.000       0.036       0.060
ar.L10                -0.0004      0.006     -0.059      0.953      -0.013       0.012
ar.L11                 0.0003      0.006      0.056      0.955      -0.012       0.012
ar.L12                -0.0143      0.006     -2.467      0.014      -0.026      -0.003
ar.L13                 0.0175      0.005      3.417      0.001       0.007       0.027
ar.L14                 0.0504      0.006      8.504      0.000       0.039       0.062
ar.L15                -0.0084      0.006     -1.499      0.134      -0.019       0.003
ar.L16                -0.0501      0.006     -8.600      0.000      -0.062      -0.039
ar.L17                 0.0076      0.005      1.393      0.164      -0.003       0.018
ar.L18                -0.0162      0.005     -2.946      0.003      -0.027      -0.005
ar.L19                -0.0351      0.006     -5.983      0.000      -0.047      -0.024
ar.L20                -0.0172      0.007     -2.628      0.009      -0.030      -0.004
ar.L21                 0.0212      0.007      3.083      0.002       0.008       0.035
ar.L22                 0.0763      0.005     14.374      0.000       0.066       0.087
ar.L23                 0.0460      0.008      5.944      0.000       0.031       0.061
ar.L24                -0.0452      0.009     -5.308      0.000      -0.062      -0.029
ma.L1                  0.2002      0.083      2.423      0.015       0.038       0.362
ma.L2                 -0.7467      0.053    -13.959      0.000      -0.852      -0.642
ma.L3                 -0.5812      0.040    -14.641      0.000      -0.659      -0.503
ma.L4                  0.0367      0.063      0.585      0.558      -0.086       0.160
ma.L5                 